## Programação Semidefinida com CVXPY

Um tutorial simples de programação semidefinida em Python a partir de exemplos usando a biblioteca CVXPY <br>
(Instruções de instalação em https://www.cvxpy.org/install/)

---
#### Primeiro exemplo

Como primeiro problema, iremos encontrar o estado que minimiza o valor esperado da energia esperada para uma dada Hamiltoniana $H$, restrito a valores esperados fixos para alguns observáveis, $O_1$ e $O_2$

o problema é então
$$\begin{align}
\mathrm{Minimizar}&\quad \mathrm{Tr}[\rho\,H] \nonumber \\
\mathrm{t.q.}&\quad \rho \succeq 0,\,\, \mathrm{Tr}[\rho] = 1 \nonumber\\
&\quad \mathrm{Tr}[\rho\,O_1] = k_1\nonumber \\
&\quad \mathrm{Tr}[\rho\,O_2] = k_2
\end{align}
$$

Onde $O_1,O_2,H$ e $\rho$ são matrizes Hermiteanas e $k_1,\,k_2\,\in \mathbb{R}$. 

In [17]:
import numpy as np
import cvxpy as cvx

# Definindo constantes para o problema

# Matrizes de Pauli
X = np.array([[0,1],[1,0]], dtype=np.complex128)
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]], dtype=np.complex128)

# Hamiltoniana
# Construímos a partir das matrizes de Pauli como (XX + ZZ)/2
H = 0.5*(np.kron(X, X) + np.kron(Z, Z))

# Observáveis -- Usando aqui IX e XI
O1 = np.kron(X, np.eye(2))
O2 = np.kron(np.eye(2), X)

# Valores das restrições
k1 = 0
k2 = 0

# Variável de otimização
# Matriz 4x4 (= 2 qbits) com flag PSD (positive semidefiniteness)
#
# A flag PSD já estabelece que ρ deve ser uma matriz positiva-semidefinida
# Outra forma de estabelecer essa restrição é adicioná-la a posteriori como
# ρ >> 0
ρ = cvx.Variable((4,4), PSD = True)

# Restrições
# Uma lista de equações e/ou desigualdades que devem ser satisfeitas pelas 
# variáveis do problema
Constr = [cvx.trace(ρ) == 1]   # Normaliização
Constr += [cvx.trace(ρ@O1) == k1] # Restrição ao valor médio de O1 a <O1> = k1
Constr += [cvx.trace(ρ@O2) == k2] # Restrição a <O2> = k2

# Função objetivo
Obj = cvx.real(cvx.trace(ρ@H))

# Criando uma instância de um problema de otimização
SDP_Prob = cvx.Problem(cvx.Minimize(Obj), Constr)

#### Resolvendo o problema

Aqui precisamos usar um solver na função solve() abaixo para resolver o problema, passado como primeiro argumento. O solver usado será o SCS, que é livre e pode ser instalado da mesma forma que o CVXPY.

O solver é quem de fato resolve o problema, o CVXPY introduz uma linguagem que simplifica a forma de descrever e montar o problema de otimização, ao mesmo tempo impondo o paradigma de Programação Convexa Disciplinada (DCP), que apesar de restringir minimamente a abrangência dos algoritmos, garante a convexidade do problema inserido.

Cada solver pode ser melhor em problemas distintos e é sempre interessante tentar diferentes solvers quando
algum solver falha em convergir para um problema em particular.

Para saber quais solvers estão instalados, use cvx.installed_solvers() <br>
Alguns solvers são comerciais e precisam de licença para funcionar. Um exemplo é o MOSEK, que oferece licença acadêmica gratuitamente. Uma vez com o solver instalado, ele se torna disponível pra uso na função solve()


In [18]:
SDP_Prob.solve('SCS', verbose = True)
print()

# Lendo a solução do problema
if SDP_Prob.status == "optimal":
    print(f'Resultado do SDP: <H> = {SDP_Prob.value}')
    print('Matriz densidade obtida:')
    print(ρ.value)
    
else:
    print('Status final do solver:')
    print(SDP_Prob.status)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jul 24 09:50:09 PM: Your problem has 16 variables, 3 constraints, and 0 parameters.
(CVXPY) Jul 24 09:50:09 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 24 09:50:09 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 24 09:50:09 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 24 09:50:09 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 24 09:50:09 PM: Reduction chain: Complex2Real -> Dcp2Cone -> CvxAttr2Constr -> ConeMa

---
#### Segundo Exemplo&#151;Otimização de uma desigualdade de Bell com estados k-extensíveis

Para um problema mais complexo, iremos otimizar a desigualdade CHSH com um estado de 2 qbits, usando medidas fixas (já sabidas serem suficientes para atingir o máximo quântico de $2\sqrt{2}$). 

Ao estender o estado, esperamos que a violação máxima diminua, uma vez que quanto maior a extensão, mais próximo o conjunto de estados se aproxima do conjunto de estados separáveis, que só permitem atingir o valor de $2$ na desigualdade.

In [25]:
# Coeficientes da desigualdade
# Bell[x,y] será o coeficiente da desigualdade para as correlações <Ax By>,
# onde <Ax By> = p(a = b|x,y) - p(a != b|x,y). No caso quântico, esse valor cor-
# responde ao valor esperado dos observáveis medidos
#
# A desigualdade usada será <A0 B0> + <A0 B1> + <A1 B0> - <A1 B1>,
# teremos Bell[x,y] = (-1)^(x*y)
Bell = np.array([[1., 1.],[1., -1.]])

#=====================================================================================
# Observáveis de Alice e Bob
# Alice
Ax = np.zeros((2,2,2))
Ax[:,:,0] = 1.*X  # Pauli X para escolha de medida x=0
Ax[:,:,1] = 1.*Z  # Pauli Z para x=1

# Bob
By = np.zeros((2,2,2))
By[:,:,0] = 1.*(X+Z)/np.sqrt(2)  # y = 0
By[:,:,1] = 1.*(X-Z)/np.sqrt(2)  # y = 1

#=====================================================================================
# Novamente a variável é ρ. Agora o tamanho depende da extensão usada
# Primeiramente resolvemos o problema sem extensão para verificar o limite
# de Tsirelson de 2*sqrt(2)
ρ = cvx.Variable((4,4), PSD = True)

# Compondo a função objetivo
Obj = 0
for x in range(2):
    for y in range(2):
        Obj += Bell[x,y]*cvx.trace(ρ@np.kron(Ax[:,:,x], By[:,:,y]))


# Restrições -- Apenas normalização no caso sem extensão
Constr = [cvx.trace(ρ) == 1]

#=====================================================================================
# Montando e resolvendo o problema
SDP_Prob = cvx.Problem(cvx.Maximize(Obj), Constr)
SDP_Prob.solve('SCS', verbose = True)
print()

# Resultados
print(f'CHSH -- Sem extensão: {SDP_Prob.value}')
print('Matriz obtida:')
print(ρ.value)  # Compare com (|00> + |11>)/sqrt(2) e note que o resultado aqui foi obtido numericamente

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jul 24 09:53:18 PM: Your problem has 16 variables, 1 constraints, and 0 parameters.
(CVXPY) Jul 24 09:53:18 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 24 09:53:18 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 24 09:53:18 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 24 09:53:18 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 24 09:53:18 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeM

C:\Users\ranie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  
C:\Users\ranie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  from ipykernel import kernelapp as app
C:\Users\ranie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ranie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:20: ComplexWarning: Casting complex values to real discards the imaginary part


#### Extensão simétrica k=1 - Sistemas A,B1,B2

Agora consideramos o estado com uma cópia da partição B. Isto é, queremos um estado $\rho_{AB}$ que seja resultado do traço parcial de um estado maior $\rho_{A B_1 B_2}$ simétrico em relação à permutação entre $B_1$ e $B_2$, i.e. tal que $\mathrm{SWAP}_{B_1 B_2}\,\rho_{A B_1 B_2}\,\mathrm{SWAP}^\dagger_{B_1 B_2} = \rho_{A B_1 B_2}$.

O problema agora tem a forma
$$\begin{align}
\mathrm{Maximizar}&\quad \sum_{x,y} \beta_{xy}\,C_{xy} \nonumber \\
\mathrm{t.q.}&\quad C_{xy} = \mathrm{Tr}[\rho_{AB}\,A_x \otimes B_y] \nonumber \\
&\quad \rho_{AB} = \mathrm{Tr}_{B_2}[\rho_{A B_1 B_2}] \nonumber \\
&\quad \rho_{A B_1 B_2} \succeq 0,\,\, \mathrm{Tr}[\rho_{A B_1 B_2}] = 1 \nonumber \\
&\quad \mathrm{SWAP}_{B_1 B_2}\,\rho_{A B_1 B_2}\,\mathrm{SWAP}^\dagger_{B_1 B_2} = \rho_{A B_1 B_2}
\end{align}
$$
onde $\beta_{xy}$ são coeficientes de uma desigualdade de Bell (usaremos a desigualdade CHSH aqui) e $A_x$ e $B_y$ ($x,\,y \in \{0,\,1\}$) são observáveis fixos correspondentes às medidas realizadas por Alice e Bob, fornecidos como parâmetros constantes para o problema. Apesar das várias etapas no programa acima, a única variável "legítima" do problema é $\rho_{A B_1 B_2}$, com as outras variáveis sendo definidas a partir de operações sobre ela que não fornecem graus de liberdade extras.

Todas as restrições são igualdades ou desigualdades matriciais lineares nas variáveis do problema, o que faz do problema uma SDP. Note que isso não seria verdade se não fornecêssemos $A_x$ e $B_y$ como parâmetros. Nesse caso, teríamos uma relação multilinear ao definir $C_{xy}$ e algum método de aproximação seria necessário para poder tratar o problema como uma SDP.


In [8]:
# Nova variável 
# Agora temos ρ estendido com subsistemas A, B1 e B2
ρ_ex = cvx.Variable((8,8), PSD = True)

#=====================================================================================
# Traço parcial sobre B2
def PTr(Rho_in):
    # Não é a maneira mais esperta de fazer o traço, mas serve pros propósitos daqui

    T0 = np.kron(np.eye(4), np.array([1,0]))
    T1 = np.kron(np.eye(4), np.array([0,1]))

    return T0@Rho_in@(T0.T) + T1@Rho_in@(T1.T)

#=====================================================================================
# Operador de troca
# Primeiro a forma bipartida: SW = sum_ij |ij><ji|
# Eye é a identidade: sum_ij |ij><ij|
# O reshape torna SW em um tensor: SW_ijij
SW = np.reshape(np.eye(4),[2,2,2,2])

# Permutação para SW_ijji e retorno ao formato 4x4
SW = np.transpose(SW, [0,1,3,2])
SW = np.reshape(SW, [4,4])

# Forma tripartida I x SW
SW = np.kron(np.eye(2), SW)

#=====================================================================================
# Compondo a função objetivo
Obj = 0
for x in range(2):
    for y in range(2):
        Obj += Bell[x,y]*cvx.trace(PTr(ρ_ex)@np.kron(Ax[:,:,x], By[:,:,y]))

# Restrições - Agora incluimos a simetria pela troca entre B1 e B2
Constr = [cvx.trace(ρ_ex) == 1]
Constr += [SW@ρ_ex@SW == ρ_ex]

#=====================================================================================
# O problema
SDP_Prob = cvx.Problem(cvx.Maximize(Obj), Constr)
SDP_Prob.solve('SCS', verbose = True)
print()

print(f'CHSH -- Extensão k=1: {SDP_Prob.value}')
print('Matriz obtida:')
print(PTr(ρ_ex.value))

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jul 24 06:06:06 PM: Your problem has 64 variables, 2 constraints, and 0 parameters.
(CVXPY) Jul 24 06:06:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 24 06:06:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 24 06:06:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 24 06:06:06 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 24 06:06:06 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeM

---
Notamos no caso anterior que a extensão pra um subsistema extra já foi suficiente para perdermos qualquer violação da desigualdade CHSH. É possível que já tenhamos um estado separável, mas também é possível que as medidas usadas não sejam úteis para revelar a não-localidade no estado obtido.

Poderíamos usar SDPs também para otimizar medidas, adicionando restrições da forma $\mathbb{1} \succeq A_x \succeq -\mathbb{1}$ e $\mathbb{1} \succeq B_y \succeq -\mathbb{1}$ sobre os observáveis (ou restrições $M^{(a)}_x \succeq 0,\,\sum_a M^{(a)}_x = \mathbb{1}$ para POVMs, onde $a$ é o valor clássico retornado pela medida). No entanto, como o problema se torna não-linear ao tratar $A_x$ e $B_y$ como variáveis simultaneamente, podemos recorrer a um método de _otimização alternada_ para encontrar uma cota inferior ao máximo global do problema.

Nesse caso, inicializamos as variáveis com algum valor e rodamos um SDP para cada variável, otimizando cada uma na sequência, fixando as outras variáveis como constantes e usando o resultado como constante na otimização seguinte. No exemplo abaixo, usaremos para $\rho$ a expressão obtida acima e otimizaremos sobre $A_x$ e $B_y$, inicializados com os valores usados no problema anterior.

Em cada iteração do método, otimizamos sobre $A_x$ fixando $B_y$ num primeiro passo, então usamos o resultado de $A_x$ como constante no passo seguinte, onde otimizamos sobre $B_y$. Com o resultado para $B_y$ informamos o valor do observável de Bob para a otimização em $A_x$ e repetimos o processo. 

De fato, como a função objetivo é a mesma em todas as etapas, o valor nunca decresce e converge para algum máximo. No entanto, não há como garantir que se trata de um máximo global ou saber quantos passos serão necessários para uma convergência. 

---
O código abaixo foi mantido para ilustrar o método. Aparentemente temos um situação onde tavez não seja possível ir além de $CHSH = 2$ e o problema se torna numericamente instável com 2 iterações da otimização


In [29]:
# Quantidade de repetições da otimização alternada
qrep = 1

# Usando o estado obtido anteriormente como parâmetro
ρ_k1 = PTr(ρ_ex.value)

# Produzindo valores iniciais para Ax e By aleatoriamente
Ax = np.zeros((2,2,2))
By = np.zeros((2,2,2))

for x in range(2):
    # Gerando os autovalores de Ax e By
    Ax[:,:,x] = 2*np.diag(np.random.rand(2)-0.5)
    By[:,:,x] = 2*np.diag(np.random.rand(2)-0.5)
        
    # Girando a base por uma unitária aleatória
    # Vamos produzir U = [ e^(iθ0)*cos(ϕ),  e^(iθ1)*sin(ϕ),
    #                     -e^(-iθ1)*sin(ϕ), e^(-iθ0)*cos(ϕ)]
    # Para distribuição uniforme, usamos ϕ = arcsin(sqrt(ξ))
    
    θ = 2*np.pi*np.random.rand(2)
    ξ = np.random.rand()
    
    U = np.sqrt(1-ξ)*(np.cos(θ[0])*np.eye(2, dtype=np.complex128) + 1j*np.sin(θ[0])*Z) 
    U += np.sqrt(ξ)*(np.cos(θ[1])*1j*Y + np.sin(θ[1])*X)
    
    Ax[:,:,x] = U@Ax[:,:,x]@(np.conj(U.T))

    θ = 2*np.pi*np.random.rand(2)
    ξ = np.random.rand()
    
    U = np.sqrt(1-ξ)*(np.cos(θ[0])*np.eye(2) + 1j*np.sin(θ[0])*Z) 
    U += np.sqrt(ξ)*(np.cos(θ[1])*1j*Y + np.sin(θ[1])*X)
    
    By[:,:,y] = U@By[:,:,x]@(np.conj(U.T))
    
# Variáveis do problema
Ax_var = [cvx.Variable((2,2)) for x in range(2)]
By_var = [cvx.Variable((2,2)) for y in range(2)]

for _ in range(qrep):
    
    # Otimização sobre observáveis de Alice
    
    Obj = 0
    for x in range(2):
        for y in range(2):
            Obj += Bell[x,y]*cvx.trace(ρ_k1@cvx.kron(Ax_var[x], By[:,:,y]))
    
    # Restrições na forma de desigualdade matricial:
    # -1 <= Ax <= 1
    # Os operadores >> e << impõem as condições. Dizer que uma matriz A precede a matriz B, A << B,
    # equivale a dizer que B - A é positivo-semidefinido, i.e. B-A >> 0
    Constr = [Ax_var[0] << np.eye(2)]
    Constr += [Ax_var[0] >> -1.*np.eye(2)]
    Constr += [Ax_var[1] << np.eye(2)]
    Constr += [Ax_var[1] >> -1.*np.eye(2)]
    
    # Resolvendo o problema para o primeiro passo
    SDP_Prob = cvx.Problem(cvx.Maximize(Obj), Constr)
    SDP_Prob.solve('SCS')

    # Fixando o valor de Ax para o passo seguinte
    for x in range(2):
        Ax[:,:,x] = Ax_var[x].value

    #=====================================================================================
    # Otimizando sobre observáveis de Bob
    # Reciclaremos algumas variáveis aqui    
    Obj = 0
    for x in range(2):
        for y in range(2):
            Obj += Bell[x,y]*cvx.trace(ρ_k1@cvx.kron(Ax[:,:,x], By_var[y]))
    
    Constr = [By_var[0] << np.eye(2)]
    Constr += [By_var[0] >> -1.*np.eye(2)]
    Constr += [By_var[1] << np.eye(2)]
    Constr += [By_var[1] >> -1.*np.eye(2)]
    
    SDP_Prob = cvx.Problem(cvx.Maximize(Obj), Constr)
    SDP_Prob.solve('SCS')
    
    for y in range(2):
        By[:,:,y] = By_var[y].value
        
print(f'\nValor final após otimização de medidas: {SDP_Prob.value}')

print(f'\n{Ax_var[0].value}\n\n{Ax_var[1].value}')
print(f'\n{By_var[0].value}\n\n{By_var[1].value}')


Valor final após otimização de medidas: 2.0000020326268455

[[1.00000378e+00 6.43035548e-07]
 [6.43004781e-07 1.00000043e+00]]

[[9.99999898e-01 1.16690562e-07]
 [1.16688135e-07 9.99999951e-01]]

[[ 1.00000000e+00  4.45342454e-04]
 [-4.45342469e-04  1.00000000e+00]]

[[ 0.00069354  0.00045654]
 [-0.00030389  0.0002012 ]]


C:\Users\ranie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ranie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: ComplexWarning: Casting complex values to real discards the imaginary part
